In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.grid_search import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import warnings
warnings.filterwarnings('ignore')

/usr/local/var/pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/var/pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
def ave(lst,interval):
    ave_lst=np.zeros((len(lst),int(len(lst[0])/6)))
    
    for i in range(0,len(lst)):
        for j in range(0,int(len(lst[0])/6)):
            ave_lst[i][j] = (sum(lst[i][j::interval])/6)
    return ave_lst



def feature_delite_outindex(x,out_index):  
    new_index = np.array(range(len(x[0])))
    index = np.array(range(len(x[0])-len(out_index)))
    for i in range(0,len(out_index)):
        new_index = np.delete(new_index, out_index[i]-i)
   
    new_x = np.zeros((len(x),len(x[0])-len(out_index)))
    for k in range(0,len(x)):      
        for i in range(0,len(new_x[0])):
            new_x[k][index[i]] = x[k][new_index[i]]
    return new_x 

def dispersion_outindex(x):
    out_index=[] 
    for i in range(0,len(x[0])):
        flag=0
        value = x[0][i]
        for j in range(0,len(x)):
            if(value == x[j][i]):
                value = x[j][i]                
            else:
                if(flag!=1):
                    flag=1
        if(flag!=1):
            out_index.append(i)

    return out_index




def feature_select(x,out_index):   
    new_index = np.array(range(len(x[0])))
    index = np.array(range(len(x[0])-len(out_index)))
    for i in range(0,len(out_index)):
        new_index = np.delete(new_index, out_index[i]-i)
   
    new_x = np.zeros((len(x),len(x[0])-len(out_index)))
    for k in range(0,len(new_x)):      
        for i in range(0,len(new_x[0])):
            new_x[k][index[i]] = x[k][new_index[i]]
    return new_x ,out_index




def nfeature(x,oi):
    new_index = np.array(range(len(x)))
    for i in range(0,len(oi)):
        new_index = np.delete(new_index, oi[i]-i)
   
    new_x = []
    for i in range(0,len(new_index)):      
        new_x.append(x[new_index[i]])
    return new_x

In [7]:
data = np.loadtxt("wang_separate_dataset.csv",delimiter=",",dtype=float)
feature = data[:,0:1236]
test_label = data[:,-1]

ave_feature = ave(feature,206)


file = open("WANGZANKI.csv","w")
name = np.loadtxt("特徴量_名前一覧.csv",delimiter=",",dtype=str)


for i in range(0,len(name)):
    file.write(name[i])
    file.write(",")
file.write("logPapp\n")

for i in range(33):
    for j in range(0,len(name)-1):
        file.write(str(ave_feature[i][j]))
        file.write(",")
    file.write(str(test_label[i]))
    file.write("\n")
file.close()

# 一気に分散0と外れ値を除く

## 全体モデルから

### まずは訓練データ内にて分散0除去

In [38]:
furukawa=np.loadtxt("CSV folder/furukawa_全体206次元_名前付き.csv",delimiter=",",dtype=str)
f_feature=furukawa[1:,:-1]
f_label=furukawa[1:,-1]

out_index = dispersion_outindex(f_feature)

### 検証データ内にて分散0除去

In [39]:
wang=np.loadtxt("CSV folder/wang_全体公式_名前付き.csv",delimiter=",",dtype=str)
w_feature=wang[1:,1:-1]
w_label=wang[1:,-1]

out_index2 = dispersion_outindex(w_feature)

### これらだけ取り除く

In [40]:
oi = out_index+out_index2
oi=list(set(oi))
oi.sort()
len(oi)

52

In [7]:
furukawa = np.loadtxt("CSV folder/furukawa_全体206次元_名前付き.csv",delimiter=",",dtype=str)
file = open("CSV folder/furukawa_全体_分散0除去.csv","w")

new_index = np.array(range(len(furukawa[0])))
index = np.array(range(len(furukawa[0])-len(oi)))
for i in range(0,len(oi)):
    new_index = np.delete(new_index, oi[i]-i)

for i in range(394):
    for j in range(len(new_index)):
        file.write(furukawa[i][new_index[j]])
        if(j==len(new_index)-1):    
            file.write("\n")
        else:
            file.write(",")
file.close()

In [8]:
wang = np.loadtxt("CSV folder/wang_全体公式_名前付き.csv",delimiter=",",dtype=str)
wang=wang[:,1:]
file = open("CSV folder/wang_全体_分散0除去.csv","w")

new_index = np.array(range(len(wang[0])))
index = np.array(range(len(wang[0])-len(oi)))
for i in range(0,len(oi)):
    new_index = np.delete(new_index, oi[i]-i)

for i in range(38):
    for j in range(len(new_index)):
        file.write(wang[i][new_index[j]])
        if(j==len(new_index)-1):    
            file.write("\n")
        else:
            file.write(",")            
file.close()

## 残基毎のモデル

### まずは訓練データ内にて分散0除去

In [35]:
furukawa=np.loadtxt("CSV folder/furukawa_残基毎_206次元_名前付き.csv",delimiter=",",dtype=str)
f_feature=furukawa[1:,:-1]
f_label=furukawa[1:,-1]

out_i = dispersion_outindex(f_feature)

### 検証データ内にて分散0除去

In [36]:
wang=np.loadtxt("CSV folder/WANG.csv",delimiter=",",dtype=str)
w_feature=wang[:,:-1]
w_label=wang[:,-1]

out_i2 = dispersion_outindex(w_feature)

### 統合

In [37]:
oi = out_i+out_i2
oi=list(set(oi))
oi.sort()
len(oi)

52

In [18]:
furukawa = np.loadtxt("CSV folder/furukawa_残基毎_206次元_名前付き.csv",delimiter=",",dtype=str)
file = open("CSV folder/furukawa_残基毎_分散0除去.csv","w")

new_index = np.array(range(len(furukawa[0])))
index = np.array(range(len(furukawa[0])-len(oi)))
for i in range(0,len(oi)):
    new_index = np.delete(new_index, oi[i]-i)

for i in range(394):
    for j in range(len(new_index)):
        file.write(furukawa[i][new_index[j]])
        if(j==len(new_index)-1):    
            file.write("\n")
        else:
            file.write(",")
file.close()

In [19]:
wang = np.loadtxt("CSV folder/WANG.csv",delimiter=",",dtype=float)


name=np.loadtxt("CSV folder/特徴量_名前一覧.csv",delimiter=",",dtype=str)
file = open("CSV folder/wang_残基毎_分散0除去.csv","w")

new_index = np.array(range(len(wang[0])))
index = np.array(range(len(wang[0])-len(oi)))
for i in range(0,len(oi)):
    new_index = np.delete(new_index, oi[i]-i)

    
    

for i in range(34):
    if(i==0):
        for j in range(len(new_index)):
            file.write(name[new_index[j]])
            if(j==len(new_index)-1):    
                file.write("\n")
            else:
                file.write(",")
    else:
        for j in range(len(new_index)):
            file.write(str(wang[i-1][new_index[j]]))
            if(j==len(new_index)-1):    
                file.write("\n")
            else:
                file.write(",")            
file.close()